In [ ]:
!pip install networkx

In [267]:
import networkx as nx
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import operator
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import platform
from pandas._libs import parsers

In [268]:
# 한글 폰트 설정

if (platform.system() == 'Windows'):
    plt.rc('font', family='NanumBarunGothic')
else:
    plt.rc('font', family='AppleGothic')

# 한글 폰트
!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic


In [325]:
raw = pd.read_csv('/content/잇몸영양제_내용_완료.csv',encoding='utf-8-sig')
df = raw.dropna()
df.head()

,Unnamed: 0,내용,all_tokens,tokens,nouns
0,0,양치질을 아무리 잘한다고 하더라도 잘못된 양치질 또는 잇몸 자체가 약한 경우에는 잇...,"['양치질', '아무리', 'ㄴ다고', '더라도', '잘못', '양치질', '또는'...","['양치질', '아무리', '잘못', '양치질', '또는', '잇몸', '약하다',...","['양치질', '잘못', '양치질', '잇몸', '잇몸', '질환', '이야기', ..."
1,1,아무리 하루 세 번 양치를 잘한다 해도 치아에는 쉽게 문제가 생기곤 하는데 얼마 전...,"['아무리', '하루', '양치', 'ㄴ다', '아도', '치아', '문제', '생...","['아무리', '양치', '치아', '쉽다', '문제', '생기다', '얼마', '...","['양치', '치아', '문제', '얼마', '칫솔', '출혈', '통증', '시작..."
2,2,나이가 드니 치아도 약해지나 봐요옛날이면 돌도 씹어 먹었을 텐데 이제는 그토록 좋아...,"['나이', '아도', '옛날', '이제', '그토록', '좋아하', '오징어', ...","['나이', '옛날', '씹다', '그토록', '좋아하다', '오징어', '씹다',...","['나이', '옛날', '오징어', '통증', '잇몸', '나이', '전반', '신..."
3,3,잇몸 영양제조금 아쉬웠던 이유 한번씩 잇몸이 욱신대는 느낌이 느껴질 때가 있어요 그...,"['잇몸', '영양제', '조금', '아쉽', '이유', '한번', '잇몸', '다...","['잇몸', '영양제', '아쉽다', '이유', '한번', '잇몸', '신다', '...","['잇몸', '영양제', '이유', '한번', '잇몸', '느낌', '느낌', '찬..."
4,4,어머니께서 예전부터 잇몸이 좋지 않아 최근에 임플란트를 예정 중에 계신데 치과에서 ...,"['어머니', '께서', '예전', '부터', '잇몸', '최근', '예정', '계...","['어머니', '예전', '잇몸', '최근', '예정', '계시다', '상태', '...","['어머니', '예전', '잇몸', '최근', '예정', '상태', '치료', '불..."


In [326]:
# 리스트가 문자열로 저장된 경우 (ex> ["['dd','aa']"])
word_list = []

for i in df['tokens']:
  i = ast.literal_eval(i)
  word_list.append(i)

In [ ]:
word_list

In [327]:
stopwords = ['르', '과', '까지', '곳', '다시금', '제현', '귀엽다', '과', '게', '때','쿠','스트라빈스키','존','사분','김현정','머지않아','팅팅','흑흑','조속히','새끼','미친년','미친놈','개새끼',
             '링','보다','경우','반면','되다','있다','자나','살다','혹시','경우','때문','정도','사항','세진','보라','다세','건희','윤호','이재명',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '이미', '돋움','같이','우짜다','어떡하다','제대로',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','그렇다','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면','쪼르르','세계인',
                   '뭔가','또한','하다','이다','되다','다능','자다','자체','서체','지나가다','잇다','같다','미치','있다','없다', '여지','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','전하다','들다','열다',
                   '두다','서다','산하','지금','주변','대상','부분','요즘','하루','마련','시간','이상','행위','고검','야마다','검찰','하루하루','우우','엄밀히','순전히','토니오','술술','몹시','영수','이전',
                   '이하','바로 가기','바로가기','제가','먹다','보이다','지나다','콜론','각','대하다','따르다','인하다','전','후','예','한','다','보다','그렇다','좋다',
                   '나다','너무','정말','내다','많다','그리다','물론','조금','다시','느끼다','없이','특히','여러','다른','이런','가지다','어떻다','통하다', '위하다','그러다','위하다',
                   '다니다','나서다','들다','열다','어떻게','막상','가다','오다','말다','계속','쉬다','혜량','인디','망개','제법','세기',
                   '무현','여쭈다','베타','되묻다','뜨다', '다녀오다','서녀','붙다','찾다','생각나다','앉다','바니','이르다','크다',
                   '그냥','그저','바로','시키다','허다','내내','해량','걸다', '이렇다','이렇게','켜다','어린이날','그날','남주','제일','지금껏',
             '진짜','끌다','받다','넣다','무지','태주','아무튼','알다','틀다','마치','부리','익히다','만들다','잡다','아주','영재','대구','서울','매우','나오다',
             '뫄다','케이','히나','이치','경허','비다','찌다','받다','나가다','일어나다','오키','일단','우왕','드리다','큰애',
             '대다','덮다','바잔','스다','들어가다','놓아두다','놓다','틀다','들다','사다','팔다','이대로','서교동','조지','치다','그다음','다음',
             '마티','파주시','태평','잘나다','모르다','알다','알고있다','주전','작대기','아이스','들뜨다','푸다','아우','쿠쿠','가장','몰다','뭐하다', '눕다', '누르다','하하','냅다',
             '존나','데리다','트다','쓰다','타임','라인','벌리다','타다','이제','위하다','다니다','한동안','저번','마이','어쩌다','사실','그렇게','이렇게',
             '해보다','해저','가세','시마','블루','부르다','야지', '아무래도', '맞다','파주','동해','랜드','스타','검찰','가요','형질','우현',
             '과연','연하다','남자친구','한번','끌어들이다','이날','코난','전혀','최소한','그나저나','레알','님프','테너',
             '연월일','동시','문인','민사','당부','모습','바라보다','경애','총비서','동지','모읍','군호','개키다','양치식물','불쑥불쑥','드래건','논란거리','나나미','버크','애니','갑바','요지경','힐러','임화','실로',
             '들리다','리츠','달랑달랑','갈수록','크윽','겟다','이따가','프리','오리','두둥실','한마음','분뇨','조다','뽀짝','하마터면','밀다','카다','한쪽','낙도','자기','아장거리다','주도','출처','현이',
             '우현','우현이','꿈꾸다','순간','갑자기','히다','분명','중생', '근데','레즈','드르르','집어넣다',
             '보통','심의','결정','이유','근처','평점','리뷰','세븐틴','제페','니즈','멜론','혹시나','대충','순응','종종','많이','외려','얼마나','이리저리','돌리다','시간부','곁들이다',
             '부풀다','금의환향','하인','의원','위장','광인','제외','바이','고인','고래','모험','스트','레스','왠지','매번','나타나다',
             '아오키','가자','데다','어린이','나눠주다','개업','괜히','나머지','최선','이것저것','바라다','바다','다가가다','가시다','최인섭','쓰이다','사리탑','엘리엇',
             '사라','정작','얘기','발전','얼마','공산당','일수','만나다','감싸다','누르다','그제서야','태용이','유투','현진','아마','가만히','그린란드','요하다','황두','맘대로','이후','착실히','난초','저렇게','이렇게',
             '트로','옛날이야기','온종일','왕비','웃어른', '장모','지폐','진실로','촛불','통역','가스','라이팅','르네','집어넣다','드르르르','르네',
             '표시','구라','진실로','저울','어쨌든','난무','어영부영','하하','르네', '냅다','와중','피카','깨닫다','디다','너와', '김선호', '황두', '김래원','연초', '오가타','금상첨화','가을비','손안','하나',
             '새끼','잡거','베리','딜리','갈래','진짜로','날아오다','날아가다','인가','역시','갯마을','차차차','두식','주술','회전','시발','아안','하도','왜냐하면','타츠미','루씨','메루','시안','젤로',
             '잠시만','아직','비바','응수','강동원','로드','데이브','기실','일주','하여튼','컴백','디비','며칠','졸리','라리','섹트','청주','어서','요시','점심때','이따','가치','새끼','여하튼','너도나도',
             '레알','마타','에그','바자','지랄','리사','저벅저벅','아까','서화','시방','사모','나다니다','호시','아월','팔룡','슬쩍','슬그머니','퀴어','위터','포커','조슈야','가요','승관','디노','만납다','꾸리',
             '에스','쿱스','맛나','드디어','굳이','이제껏','이리','이제서야','혁이','현아','이따','비록','설마','자꾸','호주','어느새','언제','에리히','화중','업태','에이치','박박','오가타','모토','반드시',
             '민규','가령','언제','졸라','레트','리버','이쁜이','재영','트라이','김혜은','다우지','스톤','그리니치','김균','플레','지니','민수','버블','원우','입면','라라','선생','부천','현진이','근년','그룹웨어','시방',
             '사바','기철','위즈','파크','한곳','들어오다','기승','강남구','개포동','논현동','임대','개원','어젯밤','거상','베리','라덴','무조건','닷새','국극','개다','세내다','냉큼','브리','루트','한몫',
             '오자','엠피','스리','날로','러스','하마','시어터','트리','솔직히','제발','세상','대뜸','혹은','어쩌면','엄청','아무것','모에','사마','부리다','갈다','브이','로그','갖다','묻히다','에디','추스르다','뚝딱','옛날',
             '수없이','아직','테라','안네','스톱','최진욱','루프','쏘다','서불','지현','캐시','그대로','라프','또는','내면','이제서야','여전히','한인','성우','레이','위풍당당','범이','임영웅','대체','도저히','구구구',
             '논현동','매트','자파','쥐불놀이','심지어','돌아오다','낟다','시발','얼떨결','후욱','양재','충주','복합','로컬','타일','수수','최고','아직','아까','기하다','치가','그럭적럭','하필','주기','난생처음','여태','대체',
             '엮다','짓다','주무','세요','엉엉','살짝','넘다','요부','어찌나','어쩌다','무려','앞날','놔주다','고야','굉장히','선천','앞쪽','가리다','아래','결국','의하다','삼성리']
#stopwords_set = set(stopwords)


In [328]:
# 빈도수 기준으로 단어 제거 위함(트위터_자연어처리_함수.ipynb 에서 만든 워드카운트 파일 기준)
count = pd.read_csv('/content/잇몸영양제_내용_count.csv',encoding='utf-8',index_col=False)
count.head()
# count.dtypes
count = count[count['count'] >= 50 ] # 빈도수 40개 이상
count_list = list(count['keyword'])
len(count_list)

73

In [329]:
cleared_list = []
li_1 = []
li_2 = []
list1 = []

# 불용어(stopwords)에 해당하는 요소 제외 -> list1
for word in word_list:
    li_1 = []
    for w in word:
        if w not in stopwords:
           li_1.append(w)
    list1.append(li_1)

# 빈도수 기준(count_list) 충족하는 요소 추출 -> cleared_list
for li in list1:
    li_2 = []
    for l in li:
        if l in count_list:
            li_2.append(l)
    cleared_list.append(li_2)

In [ ]:
cleared_list

In [330]:
cleared_list2=[]
for cleared in cleared_list:
  #cleared : [ ]
  cnt = cleared.count('폴리스')
  print('전 ',cleared)
  li_3 = []
  #c : 한 리스트 안 요소들
  idx = 0  
  for c in cleared:
    if c not in ['프로','폴리스']:
      li_3.append(c)
      idx += 1

    elif c == '프로':
      li_3.insert(idx,'프로폴리스')
      idx += 1
    
    elif c == '폴리스':
      pass
    
  cleared_list2.append(li_3)        
  print('후 ',li_3,'\n')

  cleared_list = cleared_list2

전  ['잇몸', '약하다', '잇몸', '질환', '음식', '관리', '잇몸', '잇몸', '약하다', '잇몸', '관리', '함께', '잇몸', '관리', '방법', '잇몸', '관리', '잇몸', '잇몸', '잇몸', '사용', '방법', '사용', '방법', '함께', '잇몸', '관리', '잇몸', '영양제', '함께', '섭취', '관리', '추천', '상태', '잇몸', '관리', '영양제', '잇몸', '영양제', '제품', '성분', '잇몸', '프로', '폴리스', '성분', '프로', '폴리스', '프로', '폴리스', '성분', '프로', '폴리스', '프로', '폴리스', '잇몸', '성분', '성분', '영양제', '잇몸', '영양제', '원료', '프로', '폴리스', '원료', '원료', '사용', '원료', '성분', '모두', '원료', '원료', '원료', '사용', '잇몸', '영양제', '섭취', '제품', '치아', '잇몸', '성분', '흡수', '제품', '프로', '폴리스', '칼슘', '비타민', '함유', '건강', '칼슘', '치아', '건강', '관리', '성분', '칼슘', '칼슘', '함께', '섭취', '비타민', '관리', '성분', '함께', '함께', '스테롤', '성분', '함유', '성분', '성분', '잇몸', '영양제', '생기다', '사용', '성분', '성분', '질환', '건강', '기능', '식품', '영양제', '성분', '잇몸', '영양제', '함께', '함께', '치아', '잇몸', '사용', '관리']
후  ['잇몸', '약하다', '잇몸', '질환', '음식', '관리', '잇몸', '잇몸', '약하다', '잇몸', '관리', '함께', '잇몸', '관리', '방법', '잇몸', '관리', '잇몸', '잇몸', '잇몸', '사용', '방법', '사용', '방법', '함께', '잇몸', '관리', '잇몸', '영양제', '함께', '섭취', '관리', '추천', '상태', '

In [331]:
df['cleared_token'] = cleared_list

In [332]:
df

,Unnamed: 0,내용,all_tokens,tokens,nouns,cleared_token
0,0,양치질을 아무리 잘한다고 하더라도 잘못된 양치질 또는 잇몸 자체가 약한 경우에는 잇...,"['양치질', '아무리', 'ㄴ다고', '더라도', '잘못', '양치질', '또는'...","['양치질', '아무리', '잘못', '양치질', '또는', '잇몸', '약하다',...","['양치질', '잘못', '양치질', '잇몸', '잇몸', '질환', '이야기', ...","[잇몸, 약하다, 잇몸, 질환, 음식, 관리, 잇몸, 잇몸, 약하다, 잇몸, 관리,..."
1,1,아무리 하루 세 번 양치를 잘한다 해도 치아에는 쉽게 문제가 생기곤 하는데 얼마 전...,"['아무리', '하루', '양치', 'ㄴ다', '아도', '치아', '문제', '생...","['아무리', '양치', '치아', '쉽다', '문제', '생기다', '얼마', '...","['양치', '치아', '문제', '얼마', '칫솔', '출혈', '통증', '시작...","[치아, 문제, 생기다, 함께, 치아, 관리, 관리, 치아, 약하다, 함께, 필요,..."
2,2,나이가 드니 치아도 약해지나 봐요옛날이면 돌도 씹어 먹었을 텐데 이제는 그토록 좋아...,"['나이', '아도', '옛날', '이제', '그토록', '좋아하', '오징어', ...","['나이', '옛날', '씹다', '그토록', '좋아하다', '오징어', '씹다',...","['나이', '옛날', '오징어', '통증', '잇몸', '나이', '전반', '신...","[나이, 잇몸, 약하다, 나이, 음식, 치아, 잇몸, 영양제, 섭취, 음식, 챙기다..."
3,3,잇몸 영양제조금 아쉬웠던 이유 한번씩 잇몸이 욱신대는 느낌이 느껴질 때가 있어요 그...,"['잇몸', '영양제', '조금', '아쉽', '이유', '한번', '잇몸', '다...","['잇몸', '영양제', '아쉽다', '이유', '한번', '잇몸', '신다', '...","['잇몸', '영양제', '이유', '한번', '잇몸', '느낌', '느낌', '찬...","[잇몸, 영양제, 잇몸, 생기다, 잇몸, 영양제, 챙기다, 챙기다, 생기다, 잇몸,..."
4,4,어머니께서 예전부터 잇몸이 좋지 않아 최근에 임플란트를 예정 중에 계신데 치과에서 ...,"['어머니', '께서', '예전', '부터', '잇몸', '최근', '예정', '계...","['어머니', '예전', '잇몸', '최근', '예정', '계시다', '상태', '...","['어머니', '예전', '잇몸', '최근', '예정', '상태', '치료', '불...","[잇몸, 상태, 치료, 생기다, 치아, 나이, 치아, 관리, 잇몸, 건강, 관리, ..."
...,...,...,...,...,...,...
95,95,안녕하세욤 치아는 어릴 때부터 꾸준하게 관리를 해주어야 해요평생 쓰는 치아이기 때문...,"['안녕하세요', '어리', '부터', '꾸준', '관리', '어야', '평생', ...","['어리다', '관리', '평생', '치아', '관리', '소홀', '건강', '빨...","['관리', '평생', '치아', '관리', '소홀', '건강', '빨간불', '금...","[관리, 치아, 관리, 건강, 상태, 치아, 잇몸, 잇몸, 영양제, 햇살, 햇살, ..."
96,96,잇몸 부엇을 때 잇몸 내려 앉음 걱정 잇몸 영양제를 먹어보자 지난 년간의 치아교정으...,"['잇몸', '잇몸', '내리', '걱정', '잇몸', '영양제', '지나', '년...","['잇몸', '붓다', '잇몸', '내리다', '걱정', '잇몸', '영양제', '...","['잇몸', '잇몸', '걱정', '잇몸', '영양제', '치아', '교정', '교...","[잇몸, 붓다, 잇몸, 잇몸, 영양제, 치아, 잇몸, 잇몸, 잇몸, 나이, 잇몸, ..."
97,97,부광약품 웰치부광약품부 광약품 웰치잇몸영양제 치아 영양제 치아 건강 잇몸건강 잇몸건...,"['부광약품', '웰치', '부광약품', '약품', '웰치', '잇몸', '영양제'...","['부광약품', '웰치', '부광약품', '약품', '웰치', '잇몸', '영양제'...","['부광약품', '웰치', '부광약품', '약품', '웰치', '잇몸', '영양제'...","[웰치, 웰치, 잇몸, 영양제, 영양제, 건강, 잇몸, 건강, 잇몸, 건강, 잇몸,..."
98,98,이웃님들 잇몸영양제 챙기고 있으신가요 저는 치아교정을 결혼하고 했는데 아무래도 늦은...,"['이웃', '잇몸', '영양제', '챙기', '으시', 'ㄴ가요', '치아', '...","['이웃', '잇몸', '영양제', '챙기다', '치아', '교정', '결혼', '...","['이웃', '잇몸', '영양제', '치아', '교정', '결혼', '나이', '치...","[잇몸, 영양제, 챙기다, 치아, 나이, 치아, 잇몸, 영양제, 챙기다, 바이오, ..."


In [333]:
df.to_csv('/content/잇몸영양제_내용_네트워크.csv',encoding = 'utf-8-sig')

### 동시 출현 빈도 계산

In [334]:
# 동시 출현 빈도
count = {}

for context in word_list:
    #print(context)
    for i, a in enumerate(context):
        for b in context[i+1:]:
            if a>b:
                count[b,a] = count.get((b,a),0) + 1
            else:
                count[a,b] = count.get((a,b),0) + 1

count = {key: value for key, value in count.items() if key[0] != key[1]}

### 메트릭스 만들기

In [279]:
import nltk
from nltk import ConditionalFreqDist
from nltk.util import ngrams

In [335]:
# 단어 제거한 파일 저장 후에 한번더 불러와야 코드 정상작동함 (전처리 부분에 저장코드 있음)
df = pd.read_csv('/content/잇몸영양제_내용_네트워크.csv')

In [336]:
count

{('아무리', '양치질'): 16,
 ('양치질', '잘못'): 27,
 ('또는', '양치질'): 9,
 ('양치질', '잇몸'): 708,
 ('약하다', '양치질'): 42,
 ('양치질', '질환'): 73,
 ('양치질', '피하다'): 10,
 ('양치질', '이야기'): 23,
 ('양치질', '특히나'): 14,
 ('양치질', '음식'): 56,
 ('많이', '양치질'): 68,
 ('방문', '양치질'): 11,
 ('관리', '양치질'): 212,
 ('양치', '양치질'): 26,
 ('양치질', '함께'): 103,
 ('보통', '양치질'): 25,
 ('양치질', '올바르다'): 26,
 ('양치질', '칫솔'): 30,
 ('가르치다', '양치질'): 10,
 ('방법', '양치질'): 61,
 ('양치질', '여럿'): 5,
 ('양치질', '우선'): 10,
 ('마사지', '양치질'): 15,
 ('아기', '양치질'): 5,
 ('양치질', '어렵다'): 16,
 ('양치질', '유치'): 5,
 ('거즈', '양치질'): 5,
 ('양치질', '이용'): 37,
 ('방식', '양치질'): 9,
 ('싣다', '양치질'): 12,
 ('사용', '양치질'): 80,
 ('양치질', '영양제'): 398,
 ('섭취', '양치질'): 122,
 ('양치질', '해주'): 5,
 ('양치질', '추천'): 25,
 ('받았다', '양치질'): 6,
 ('양치질', '주기'): 21,
 ('검진', '양치질'): 31,
 ('양치질', '필수'): 22,
 ('상태', '양치질'): 49,
 ('안내', '양치질'): 5,
 ('돌아오다', '양치질'): 6,
 ('생활', '양치질'): 8,
 ('습관', '양치질'): 32,
 ('돌아보다', '양치질'): 5,
 ('알아보다', '양치질'): 39,
 ('시작', '양치질'): 29,
 ('알려지다', '양치질'): 11,
 ('대표', '양치질'): 13,
 ('양치질

In [337]:
df['cleared_token']

0     ['잇몸', '약하다', '잇몸', '질환', '음식', '관리', '잇몸', '잇...
1     ['치아', '문제', '생기다', '함께', '치아', '관리', '관리', '치...
2     ['나이', '잇몸', '약하다', '나이', '음식', '치아', '잇몸', '영...
3     ['잇몸', '영양제', '잇몸', '생기다', '잇몸', '영양제', '챙기다',...
4     ['잇몸', '상태', '치료', '생기다', '치아', '나이', '치아', '관...
                            ...                        
95    ['관리', '치아', '관리', '건강', '상태', '치아', '잇몸', '잇몸...
96    ['잇몸', '붓다', '잇몸', '잇몸', '영양제', '치아', '잇몸', '잇...
97    ['웰치', '웰치', '잇몸', '영양제', '영양제', '건강', '잇몸', '...
98    ['잇몸', '영양제', '챙기다', '치아', '나이', '치아', '잇몸', '...
99    ['플러스', '프리미엄', '잇몸', '영양제', '플러스', '프리미엄', '잇...
Name: cleared_token, Length: 100, dtype: object

In [338]:
# 빈문자열 확인 코드
for li in word_list:
  if not li:
    print(li)
    del li[:]

In [339]:
words = []
bigram = []
token =[]

# 빈 리스트 제거 후에 처리 시작(안하면 token += 부분에서 오류)
df = df.replace('[]',np.nan)
df = df.dropna(how = 'any')

for i in df['cleared_token']:
    #print(i)
    i = ast.literal_eval(i)
    words.append(i)

# # 단어 두 개씩 조합
for word in words:
    #print(word)
    bigrams = ngrams(word,2)
    bigram.append(bigrams)

for i in bigram:
  token += ([x for x in i])

cfd = ConditionalFreqDist(token)
cfd.conditions()

['잇몸',
 '약하다',
 '질환',
 '음식',
 '관리',
 '함께',
 '방법',
 '사용',
 '영양제',
 '섭취',
 '추천',
 '상태',
 '제품',
 '성분',
 '프로폴리스',
 '원료',
 '모두',
 '치아',
 '흡수',
 '칼슘',
 '비타민',
 '함유',
 '건강',
 '스테롤',
 '생기다',
 '기능',
 '식품',
 '문제',
 '필요',
 '치료',
 '치석',
 '챙기다',
 '영양',
 '도움',
 '선택',
 '구강',
 '형성',
 '염증',
 '항산화',
 '억제',
 '유해',
 '유지',
 '함량',
 '작용',
 '신경',
 '꾸준히',
 '나이',
 '화질',
 '옵션',
 '자동',
 '케어',
 '치약',
 '추출물',
 '치주',
 '붓다',
 '프리미엄',
 '유일',
 '아보카도',
 '국내',
 '선물',
 '포장',
 '아프다',
 '부모',
 '햇살',
 '플러스',
 '인사',
 '유산균',
 '바이오',
 '리얼',
 '웰치']

In [340]:
len(token)

10693

In [341]:
freq_matrix = []

for i in cfd.keys():
  temp = []
  for j in cfd.keys():
    temp.append(cfd[i][j])
  freq_matrix.append(temp)
freq_matrix = np.array(freq_matrix)

df = pd.DataFrame(freq_matrix, index = cfd.keys(), columns = cfd.keys())
#df.style.background_gradient(cmap='coolwarm')

In [342]:
df

,잇몸,약하다,질환,음식,관리,함께,방법,사용,영양제,섭취,...,포장,아프다,부모,햇살,플러스,인사,유산균,바이오,리얼,웰치
잇몸,122,43,37,8,43,6,1,6,614,4,...,0,19,6,0,1,6,0,15,0,2
약하다,23,0,1,1,1,2,0,0,0,0,...,0,2,2,0,0,0,0,0,0,1
질환,24,0,2,1,4,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
음식,13,0,0,6,2,2,2,0,0,6,...,0,1,1,0,0,0,0,0,0,0
관리,57,2,0,2,9,6,9,2,6,3,...,1,3,2,3,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
인사,8,0,0,0,1,1,0,0,0,1,...,0,0,0,2,45,3,0,0,0,0
유산균,11,0,0,0,2,1,1,0,0,5,...,1,0,0,0,0,0,17,6,1,0
바이오,2,0,0,0,0,0,0,0,2,0,...,0,6,0,0,0,0,0,1,43,0
리얼,4,0,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,1,0,1,0


In [343]:
df.to_csv('/content/잇몸영양제_matrix.csv', encoding = 'utf-8-sig')